<a href="https://colab.research.google.com/github/coldbilberry/repo-gui/blob/main/%D0%A3%D1%80%D0%BE%D0%BA_5_%D0%A1%D0%B5%D0%B3%D0%BC%D0%B5%D0%BD%D1%82%D0%B0%D1%86%D0%B8%D1%8F_%D0%B8%D0%B7%D0%BE%D0%B1%D1%80%D0%B0%D0%B6%D0%B5%D0%BD%D0%B8%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import torchvision.transforms as transforms

from PIL import Image
from torch.nn import functional as F

from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
csv_path = '/content/drive/My\ Drive/set-lipstick-original/list.csv'
img_path = '/content/drive/My\ Drive/set-lipstick-original/720p/'
mask_path = '/content/drive/My\ Drive/set-lipstick-original/mask/'

In [4]:
class CustomDatasetFromImages(Dataset):
    def __init__(self, data_info: pd.DataFrame, img_path, mask_path, transform=None):
        self.data_info = data_info
        self.data_info['filename'] = self.data_info['filename'].apply(lambda f: os.path.join(img_path, f) if os.path.exists(os.path.join(img_path, f)) else np.NaN)
        self.data_info['mask'] = self.data_info['mask'].apply(lambda f: os.path.join(mask_path, f) if os.path.join(mask_path, f) else np.NaN)
        self.data_info.dropna(axis=0, inplace=True)

        self.img_path = [os.path.join(img_path, image_id) for image_id in sorted(os.listdir(img_path))]
        self.mask_path = [os.path.join(mask_path, image_id) for image_id in sorted(os.listdir(mask_path))]
        self.data_len = len(self.data_info)
        self.transform = transform

    def __getitem__(self, index):

        # read images and masks
        image = cv2.cvtColor(cv2.imread(str(self.data_info.iloc[index]['filename'])), cv2.COLOR_BGR2RGB)
        mask = cv2.cvtColor(cv2.imread(str(self.data_info.iloc[index]['mask'])), cv2.COLOR_BGR2RGB)

        image = cv2.resize(image, (256, 256))
        mask = cv2.resize(mask, (256, 256))
        mask = mask / 255

        image = image.astype('float32')
        mask = mask.astype('float32')

        if self.transform:
            image, mask = self.transform(image, mask)

        return transforms.ToTensor()(image.astype('float32')), transforms.ToTensor()(mask.astype('float32'))

    def __len__(self):
        return self.data_len

In [ ]:
X_train, X_test = train_test_split(df,test_size=0.3)

X_train.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)

train_data = CustomDatasetFromImages(X_train,img_path,mask_path)
test_data = CustomDatasetFromImages(X_test,img_path,mask_path)

In [ ]:
train_data_loader = DataLoader(train_data,batch_size=1,shuffle = True)
test_data_loader = DataLoader(test_data,batch_size=1,shuffle = False)

for img, mask in train_data_loader:
    print(img.shape)
    print(mask.shape)
    plt.imshow(np.array(img[0].permute(1,2,0),np.int32))
    plt.show()
    plt.imshow(mask[0].permute(1,2,0))
    plt.show()

    break

In [ ]:
data = next(iter(train_data_loader))
print(data[0].mean(), data[0].std(), data[0].shape)

In [10]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [11]:
!pip install segmentation_models_pytorch > /dev/null

In [12]:
import segmentation_models_pytorch as smp
from segmentation_models_pytorch import Linknet

model = Linknet(encoder_name='resnet50',
               encoder_weights='imagenet',
               classes=1,
               activation='sigmoid').to(device)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:03<00:00, 28.4MB/s]


In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.005)

criterion = smp.utils.losses.DiceLoss()
metric = [smp.utils.metrics.IoU(),]

In [ ]:
import segmentation_models_pytorch.utils as smp_utils

train_epoch = smp_utils.train.TrainEpoch(model=model,
                                         loss=criterion,
                                         metrics=metric,
                                         optimizer=optimizer,
                                         device=device,
                                         verbose=True)

valid_epoch = smp_utils.train.ValidEpoch(model=model,
                                         loss=criterion,
                                         metrics=metric,
                                         device=device)

In [15]:
epochs = 3
max_score = 0
PATH_MODEL = './best_model.pth'
PATH_WEIGHTS = './best_weights.pth'
score_break_value = 0.9

for epoch in range(1,epochs+1):
    print(f'Current epoch - {epoch} from {epochs}')
    train_score = train_epoch.run(train_data_loader)
    valid_score = valid_epoch.run(test_data_loader)
    print(f'Next step.\n\n')

    if max_score < valid_score['iou_score']:
        max_score = valid_score['iou_score']
        torch.save(model.state_dict(), PATH_WEIGHTS)
        torch.save(model, PATH_MODEL)
        print('Модель сохранена!')

    if max_score > score_break_value:
        break

Current epoch - 1 from 3


NameError: ignored